In [1]:
import pandas as pd
from tsnn.data_utils import *
from tsnn.models import DeepRecurrent, LSTNet, DeepSense

Using TensorFlow backend.


In [3]:
# Prediction task
input_cols = []
target_cols = ["client_0", "client_6", "client_11", "client_36", "client_315"]
pred_delay = 24

# Hyperparameters
scaling_method = "maxabs"
timesteps = 168
batch_size = 64
sampling_step = 1

# Simple cross-validation
train_ratio = 0.6
val_ratio = 0.2

In [4]:
# Load data
raw_data = pd.read_csv('electricity.txt', header=None)
raw_data.columns = ["client_" + str(i) for i in range(len(raw_data.columns))] 
nb_features = max(len(input_cols), len(raw_data.columns))

# Converting column names to their indices (necessary for LSTNet only)
target_cols_indices = colnames_to_colindices(interest_cols=target_cols, original_df=raw_data)

In [5]:
# Prepare data generators
generators_dict, stats_df = prepare_data_generators(raw_data=raw_data, 
                                                    input_cols=input_cols, 
                                                    target_cols=target_cols,     # We use the column NAMES here
                                                    scaling_method="maxabs", 
                                                    samples_length=timesteps, 
                                                    pred_delay=pred_delay,
                                                    sampling_step=sampling_step, 
                                                    batch_size=batch_size, 
                                                    train_ratio=train_ratio, 
                                                    val_ratio=val_ratio)

train_gen, train_gen_steps = generators_dict["train"]
val_gen, val_gen_steps = generators_dict["val"]
test_gen, test_gen_steps = generators_dict["test"]

# LSTNet

In [5]:
# Create model
lstnet = LSTNet(input_shape=(timesteps, nb_features), 
                interest_vars=target_cols_indices,     # We use the column INDICES here ## A CORRIGER (partial inputs)
                cnn_filters=100, 
                cnn_kernel_height=6, 
                cnn_activation='relu', 
                cnn_use_bias=True,
                gru_units=100, 
                gru_activation='relu', 
                gru_use_bias=True,
                gru_skip_units=5, 
                gru_skip_step=24, 
                gru_skip_activation='relu', 
                gru_skip_use_bias=True,
                ar_window=24, 
                ar_use_bias=True, 
                dropout=0.2)
lstnet.compile(loss='mae', optimizer='adam')

In [6]:
# Train the model
lstnet.fit_generator(generator=train_gen, 
                     steps_per_epoch=train_gen_steps, 
                     validation_data=val_gen, 
                     validation_steps=val_gen_steps,
                     epochs=3, 
                     shuffle=False)

Epoch 1/3
245/245 [==============================] - 156s - loss: 0.1232 - val_loss: 0.1081
Epoch 2/3
245/245 [==============================] - 154s - loss: 0.1034 - val_loss: 0.0965
Epoch 3/3
245/245 [==============================] - 154s - loss: 0.0916 - val_loss: 0.0883


In [12]:
# Predict on test set
pred_test_gen = yield_inputs_only(test_gen)
lstnet_preds = lstnet.predict_generator(generator=pred_test_gen, steps=test_gen_steps)
print(lstnet_preds.shape)

(5222, 5)


# DeepRecurrent

In [6]:
# Create model
deeprec = DeepRecurrent(input_shape=(timesteps,nb_features),
                        rec_layers_dims=[64,32], 
                        rec_layer_type="lstm", 
                        rec_use_bias=True, 
                        rec_activation='tanh',
                        output_dim=5, 
                        dense_use_bias=True, 
                        dropout=0.2, 
                        attention_dim="timesteps")
deeprec.compile(loss='mae', optimizer='adam')

In [7]:
# Train the model
deeprec.fit_generator(generator=train_gen, 
                      steps_per_epoch=train_gen_steps, 
                      validation_data=val_gen, 
                      validation_steps=val_gen_steps,
                      epochs=3, 
                      shuffle=False)

Epoch 1/3
245/245 [==============================] - 225s - loss: 0.1666 - val_loss: 0.1635
Epoch 2/3
245/245 [==============================] - 228s - loss: 0.1519 - val_loss: 0.1467
Epoch 3/3
245/245 [==============================] - 228s - loss: 0.1511 - val_loss: 0.1462


In [8]:
# Predict on test set
pred_test_gen = yield_inputs_only(test_gen)
deeprec_preds = deeprec.predict_generator(generator=pred_test_gen, steps=test_gen_steps)
print(deeprec_preds.shape)

(5222, 5)


# DeepSense

In [13]:
# Prepare data generators for DeepSense
deepsense_timesteps = 1000
ds_generators_dict, ds_stats_df = prepare_data_generators(raw_data=raw_data, 
                                                    input_cols=input_cols, 
                                                    target_cols=target_cols,     # We use the column NAMES here
                                                    scaling_method="maxabs", 
                                                    samples_length=deepsense_timesteps, 
                                                    pred_delay=pred_delay,
                                                    sampling_step=sampling_step, 
                                                    batch_size=batch_size, 
                                                    train_ratio=train_ratio, 
                                                    val_ratio=val_ratio)

ds_train_gen, ds_train_gen_steps = ds_generators_dict["train"]
ds_val_gen, ds_val_gen_steps = ds_generators_dict["val"]
ds_test_gen, ds_test_gen_steps = ds_generators_dict["test"]

In [14]:
# Create model
deepsense = DeepSense(sensor_dims_list=[321],
                      sequence_length=deepsense_timesteps,
                      time_window_tau=8,
                      freq_domain=False,
                      cnn_filters=64,
                      cnn1_kernel_height=2,
                      cnn2_kernel_size=3,
                      cnn3_kernel_size=2,
                      cnn4_kernel_height=2,
                      cnn5_kernel_size=3,
                      cnn6_kernel_size=2,
                      cnn_activation='relu',
                      cnn_use_bias=True,
                      gru_units=32,
                      gru_use_bias=True,
                      gru_activation='relu',
                      dropout=0.1,
                      output_dim=5)
deepsense.compile(loss='mae', optimizer='adam')

In [ ]:
# Train the model
deepsense.fit_generator(generator=ds_train_gen, 
                      steps_per_epoch=ds_train_gen_steps, 
                      validation_data=ds_val_gen, 
                      validation_steps=ds_val_gen_steps,
                      epochs=3, 
                      shuffle=False)